<a href="https://colab.research.google.com/github/MichelNivard/Boulder_hail/blob/main/04_inferring_population_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Institute for Behavioral Genetics International Statistical Genetics 2021 Workshop 

## Inferring Population Labels

Learning objectives:

1. Plot principal components obtained by running PCA on our dataset.
2. Use principal components to identify population clusters.
3. Reidentify populations for samples with missing populations using population clusters

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-memory 6G pyspark-shell'

import hail as hl
hl.plot.output_notebook()
hl.init()

Loading BokehJS ...

23/03/10 08:04:56 WARN Utils: Your hostname, ip-10-0-201-238 resolves to a loopback address: 127.0.1.1; using 10.0.201.238 instead (on interface ens5)
23/03/10 08:04:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/10 08:04:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/10 08:04:56 WARN Hail: This Hail JAR was compiled for Spark 3.3.0, running with Spark 3.3.2.
  Compatibility is not guaranteed.
23/03/10 08:04:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.3.2
SparkUI available at http://10.0.201.238:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b123465fc0bf
LOGGING: writing to /home/timp/2023_IBG_HAIL/2021_IBG_Hail/hail-20230310-0804-0.2.109-b123465fc0bf.log


## Read in QC'ed data and PCA scores

First, we'll need to read back in the sample annotations and the PCA scores from the previous practical.

In [ ]:
pca_scores = hl.read_table('resources/pca_scores.ht')

In [ ]:
sa = hl.import_table('resources/1kg_annotations.txt', 
                     impute=True, 
                     key='Sample')

FatalError: HailException: arguments refer to no files: Vector(resources/1kg_annotations.txt).

Java stack trace:
is.hail.utils.HailException: arguments refer to no files: Vector(resources/1kg_annotations.txt).
	at is.hail.utils.ErrorHandling.fatal(ErrorHandling.scala:17)
	at is.hail.utils.ErrorHandling.fatal$(ErrorHandling.scala:17)
	at is.hail.utils.package$.fatal(package.scala:78)
	at is.hail.expr.ir.StringTableReader$.getFileStatuses(StringTableReader.scala:41)
	at is.hail.expr.ir.StringTableReader$.apply(StringTableReader.scala:29)
	at is.hail.expr.ir.StringTableReader$.fromJValue(StringTableReader.scala:35)
	at is.hail.expr.ir.TableReader$.fromJValue(TableIR.scala:113)
	at is.hail.expr.ir.IRParser$.table_ir_1(Parser.scala:1582)
	at is.hail.expr.ir.IRParser$.$anonfun$table_ir$1(Parser.scala:1553)
	at is.hail.utils.StackSafe$More.advance(StackSafe.scala:64)
	at is.hail.utils.StackSafe$.run(StackSafe.scala:16)
	at is.hail.utils.StackSafe$StackFrame.run(StackSafe.scala:32)
	at is.hai

Let's randomly throw away some of our population information.

In [ ]:
sa = sa.annotate(
    SuperPopulation = hl.if_else(
        hl.rand_bool(0.9),
        sa.SuperPopulation,
        hl.missing(hl.tstr)
    )
)
sa.write('output/censored_1kg_annotations.txt', overwrite=True)
sa = hl.read_table('output/censored_1kg_annotations.txt')

Now, we'll take the first 4 PCs from the PCA table, and add the population information for each sample from our dataset.

In [ ]:
ht = pca_scores.select(PC1=pca_scores.scores[0],
                       PC2=pca_scores.scores[1],
                       PC3=pca_scores.scores[2],
                       PC4=pca_scores.scores[3])
ht = ht.annotate(pheno = sa[ht.s])

The five populations present in this dataset are `AFR`, `AMR`, `EAS`, `EUR`, and `SAS`. They are three-letter codes from the 1000 Genomes project denoting the [super population of each sample](https://www.internationalgenome.org/category/population/).

## Visualize!

Let's plot all combinations of the first three principal components (PCs) against one another. Perhaps we can identify clear cluster boundaries.

In [ ]:
import bokeh

p12 = hl.plot.scatter(ht.PC1, ht.PC2, xlabel='PC1', ylabel='PC2', label=ht.pheno.SuperPopulation, size=3, width=400, height=400)
p13 = hl.plot.scatter(ht.PC1, ht.PC3, xlabel='PC1', ylabel='PC3', label=ht.pheno.SuperPopulation, size=3, width=400, height=400)

p23 = hl.plot.scatter(ht.PC2, ht.PC3, xlabel='PC2', ylabel='PC3', label=ht.pheno.SuperPopulation, size=3, width=400, height=400)

hl.plot.show(bokeh.layouts.gridplot([[p12],
                                     [p13, p23]]))

## Reidentify samples with missing ancestry based on PCA scores

Now that we can see how the populations are decomposed by the PCs, let's try to reidentify the masked samples.

First, we'll define a grading scheme to check against the true populations of each masked sample. (The `check` function will see how many masked samples you have correctly identified.)

In [ ]:
true_labels = hl.import_table('resources/true_pops.txt', key='s').cache()
def check(ht):
    ht = ht.annotate(true_pop = true_labels[ht.s].RealSuperPopulation)
    c = ht.aggregate(hl.agg.filter(hl.is_missing(ht.pheno.SuperPopulation), 
                                   hl.agg.counter((ht.unmasked, ht.true_pop))))
    n_correct = sum(count for k, count in c.items() if k[0] == k[1])
    n_wrong = sum(count for k, count in c.items() if k[0] != k[1])
    print(f'Correctly identified {n_correct} / {n_correct + n_wrong} masked samples.')
    print()
    
    for (unm, true), n in c.items():
        if unm != true:
            if unm is not None:
                print(f'Incorrectly assigned {n} {true} samples as {unm}.')
            else:
                print(f'Left {n} {true} samples unassigned.')

## Fill in the below

Your job is to expand the below code to reidentify the population labels. One of the populations has already been provided as an example.

### `case().when()` in Hail

The `case` / `when` / `default` motif you see below is a nice way to write `if` / `else if` / `else`. The returned `unmasked` will be equal to the result of the first `when` whose predicate is `True`.

### A note on `&` and `|`

Python uses `and` and `or` for logical operators. Hail expressions use `&` for 'and' and `|` for or.

This can lead to some confusion, especially since `&` and `|` often don't play nicely with expressions involving `>`, `<`, `==`, or `!=`. If both of these operators appear, you will need to wrap the comparison in parentheses.

Suppose we want to write code that returns true when "PC1 is greater than 0.1 or PC2 is less than 0.2":

**correct**:

```
(ht.PC1 > 0.1) | (ht.PC2 < 0.2)
```

**incorrect**:
```
ht.PC1 > 0.1 or ht.PC2 < 0.2
ht.PC1 > 0.1 | ht.PC2 < 0.2
(ht.PC1 > 0.1) or (ht.PC2 < 0.2)
```

### `hl.all` and `hl.any`

You might also find it easier to use `hl.all` (which is "and") and `hl.any` (which is "or"). For example, this

```
(ht.PC1 > 0.1) | (ht.PC2 < 0.2) | (ht.PC3 >= 0.1)
```

could also be written as

```
hl.any(ht.PC1 > 0.1,
       ht.PC2 < 0.2,
       ht.PC3 >= 0.1)
```

### To think about

Which population is hardest to reidentify? Why?

### Extras

Try plotting the PCs again with the re-identified population labels.

Try plotting the PCs again, highlighting the ones that you missed. (The true population labels are in the table `true_labels`.)

In [ ]:
unmasked = ht.annotate(
    unmasked = hl.case()
        .when((ht.PC2 > 0.2) & (ht.PC2 > 0.2), 'EAS')
#         .when(..., 'AFR')
#         .when(..., 'AMR')
#         .when(..., 'EUR')
#         .when(..., 'SAS')
        .default(ht.pheno.SuperPopulation)
)
check(unmasked)